In [3]:
import pandas as pd
import numpy as np
import re
import requests
import time

from bs4 import BeautifulSoup

def moveSibling(tag, number):
	i = 1
	while i <= number:
		tag = tag.nextSibling
		i += 1
	return tag

# 

In [128]:
columns_dl = 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A'
downloaded_df = pd.DataFrame(data=None, columns=columns_dl)

In [129]:
for year in range(2006, 2017):
    y1 = str(year)[2:]
    y2 = str(year+1)[2:]
    url = "http://www.football-data.co.uk/mmz4281/" + y1 + y2 + "/D1.csv"
    df = pd.read_csv(url)
    df = df.loc[:, columns_dl]
    downloaded_df = downloaded_df.append(df)

In [132]:
seasonlist = [2006 + (i+1)/(9*34) for i in range(len(downloaded_df))]
gamedaylist = [1 + np.mod(i/9,34) for i in range(len(downloaded_df))]

In [133]:
downloaded_df.loc[:, "Season"] = seasonlist
downloaded_df.loc[:, "Gameday"] = gamedaylist

In [134]:
teamlist = downloaded_df.loc[:, "HomeTeam"].unique()
data = np.array([range(len(teamlist)), teamlist]).T
team_df = pd.DataFrame(data=data, columns=['TID', 'team'])

In [135]:
cleaned_df = pd.merge(team_df, downloaded_df, how='inner', left_on='team', right_on='HomeTeam')
cleaned_df = pd.merge(team_df, cleaned_df, how='inner', left_on='team', right_on='AwayTeam', suffixes=('_A', '_H'))

columns_cl = ['Season', 'Gameday', 'TID_H', 'TID_A', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A']
cleaned_df = cleaned_df.loc[:, columns_cl].sort_values(['Season', 'Gameday'])
cleaned_df.index = range(len(cleaned_df))

cleaned_df.to_csv('Bundesliga_Data_2006_2016.csv', index=False)